In [1]:
import pandas as pd
import numpy as np
from collections import Counter
from itertools import combinations
import primefac

# Riddler Classic Problem

From Charlie Cordova comes a puzzle that brings logic and number theory to the lottery:   
    
Five friends with a lot in common are playing the Riddler Lottery, in which each must choose exactly five numbers from 1 to 70. After they all picked their numbers, the first friend notices that no number was selected by two or more friends. Unimpressed, the second friend observes that all 25 selected numbers are composite (i.e., not prime). Not to be outdone, the third friend points out that each selected number has at least two distinct prime factors. After some more thinking, the fourth friend excitedly remarks that the product of selected numbers on each ticket is exactly the same. At this point, the fifth friend is left speechless. (You can tell why all these people are friends.)
  
What is the product of the selected numbers on each ticket?  
  
Extra credit: How many different ways could the friends have selected five numbers each so that all their statements are true?

# Solution

Initiate numbers from 1 to 70.

In [2]:
nums = list(range(1,71))
len(nums)

70

# Part 1: Filter in Qualified Numbers

## Condition 1: All the numbers are not prime

In [3]:
# Test if a number is a prime.
def isPrime(n):
    if n <= 1:
        return False
    else:
        if n % 2 == 0 and n != 2:
            return False
        else:
            for i in range(2, n//2):
                if (n % i) == 0:
                    return False
            return True

Get all the prime numbers from 1 to 70.

In [4]:
primes = [n for n in nums if isPrime(n) == True]
print(f'There are {len(primes)} prime numbers from 1 to 70.')
primes

There are 19 prime numbers from 1 to 70.


[2, 3, 5, 7, 11, 13, 17, 19, 23, 29, 31, 37, 41, 43, 47, 53, 59, 61, 67]

There are 51 numbers left after kicking out these 19 prime numbers.

In [5]:
nums_filtered_1 = [n for n in nums if n not in primes]
len(nums_filtered_1)

51

## Condition 2: All the numbers have at least 2 distinct prime factors

Find numbers that only have 1 distinct prime factors.

Find all the prime factors that could get under 70 if squared.  
For example, 11 is not qualified, since $11^2$ = 121 > 70.

In [6]:
n = 1
while n**2 <= 70:
    n+=1
possible_prime_factors = [j for j in range(2, n) if isPrime(j) == True]
possible_prime_factors

[2, 3, 5, 7]

Find numbers with only 1 prime factor.

In [7]:
nums_w_only_one_prime_factor = [1]
for i in possible_prime_factors:
    power = 2
    while i ** power <= 70:
        nums_w_only_one_prime_factor.append(i ** power)
        power += 1
print(nums_w_only_one_prime_factor)

[1, 4, 8, 16, 32, 64, 9, 27, 25, 49]


There are 41 numbers left after kicking out these 10 numbers with only 1 prime factor.

In [8]:
nums_filtered_2 = [n for n in nums_filtered_1 if n not in nums_w_only_one_prime_factor]
len(nums_filtered_2)

41

## Condition 3: Kick out numbers with prime factors shown up for < 5 times

Intuition behind this filter is: There are 5 tickets, so a prime factor has to appear for five times, otherwise there will be no 5 identical products out of these 5 tickets.

Below are the prime numbers from 1 to 70.

In [9]:
primes

[2, 3, 5, 7, 11, 13, 17, 19, 23, 29, 31, 37, 41, 43, 47, 53, 59, 61, 67]

Identify prime factors that show up less than 5 times.  
For example, 13 is qualified since there are only 4 numbers with prime factor 13 under 70 (26, 39, 52, 65).

In [10]:
failed_primes = []
for p in primes:
    base = 2
    while p * base in nums:
        base += 1
    if base-1 < 6:
        failed_primes.append(p)
failed_primes

[13, 17, 19, 23, 29, 31, 37, 41, 43, 47, 53, 59, 61, 67]

Generate numbers that need to get kicked out.

In [11]:
kick_out_list = [p*i for p in failed_primes for i in range(1,6) if p*i <=70 and p*i != p]
kick_out_list

[26, 39, 52, 65, 34, 51, 68, 38, 57, 46, 69, 58, 62]

There are 28 numbers left after kicking out these 13 numbers above.

In [12]:
nums_filtered_3 = [n for n in nums_filtered_2 if n not in kick_out_list]
len(nums_filtered_3)

28

# Part 2: Find Product of the Selected Numbers on Each Ticket

Below are the numbers that survived from the previous 3 filters.

In [13]:
str(nums_filtered_3)

'[6, 10, 12, 14, 15, 18, 20, 21, 22, 24, 28, 30, 33, 35, 36, 40, 42, 44, 45, 48, 50, 54, 55, 56, 60, 63, 66, 70]'

## Step 1: Get all the combinations

Get all combinations of the 25-number set out of these 28 numbers.  
There are 3276 of them.

In [14]:
combs = [comb for comb in list(combinations(nums_filtered_3, 25))]
len(combs)

3276

In [15]:
# Create a function that get the product of all the numbers.
def get_product(x):
    base = 1
    for n in list(x):
        base *= n
    return base

## Step 2: Use a filter to remove disqualified combinations

Build a dataframe and get the products of all the 3276 combinations.

In [16]:
df = pd.DataFrame()
df['comb'] = combs
df['product'] = df['comb'].apply(lambda x: get_product(x))
df.head()

,comb,product
0,"(6, 10, 12, 14, 15, 18, 20, 21, 22, 24, 28, 30...",1679394477268123215357542400000000000
1,"(6, 10, 12, 14, 15, 18, 20, 21, 22, 24, 28, 30...",1763364201131529376125419520000000000
2,"(6, 10, 12, 14, 15, 18, 20, 21, 22, 24, 28, 30...",1847333924994935536893296640000000000
3,"(6, 10, 12, 14, 15, 18, 20, 21, 22, 24, 28, 30...",1959293556812810417917132800000000000
4,"(6, 10, 12, 14, 15, 18, 20, 21, 22, 24, 28, 30...",1889318786926638617277235200000000000


66, 55, 44, 33, 22 gotta be in it. If we don't select them, there would be only 23 numbers.

In [17]:
df['div_five_11s'] = df['product'].apply(lambda x: 1 if x % (22 * 33 * 44 * 55 * 66) == 0 else 0)
df['div_five_11s'].sum()

1771

5 numbers that can be divided by 7 should be included but there can't be 6 of them.

In [18]:
df['div_five_7s'] = df['product'].apply(lambda x: 1 if x % (7 ** 5) == 0 and x % (7 ** 6) !=0 else 0)
df['div_five_7s'].sum()

56

Apply the filter, and it comes down to 56 combinations of 25 numbers.

In [19]:
df_filtered = df[(df['div_five_7s'] == 1) & (df['div_five_11s'] == 1)]
df_filtered.shape

(56, 4)

## Step 3: Get prime factor counts for the remaining combinations

Now, build a dictionary that counts the number of occurence for each prime factor.

In [20]:
df_filtered['prime_factors'] = df_filtered['product'].apply(lambda x: Counter(list(primefac.primefac(x))))

/Users/xulian/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [21]:
factors = [2, 3, 5, 7, 11]

Create a function that examines if the count for each prime factor can be divided by 5.

In [22]:
def five_on_five(d, factors):
    flag = 0
    for f in factors:
        if d[f] % 5 != 0:
            flag = 1
    if flag == 0:
        return 1
    else: 
        return 0

d = {2: 20, 3: 20, 5: 10, 7: 5, 11: 5}
five_on_five(d, factors)

1

In [23]:
df_filtered['5_on_5'] = df_filtered['prime_factors'].apply(lambda x: five_on_five(x, factors))
df_filtered['5_on_5'].sum()

/Users/xulian/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


1

So, there is only one combination that is qualified.

Finally, get the product of the selected numbers on each ticket.

In [24]:
ans = df_filtered[df_filtered['5_on_5'] == 1] 
int(ans['product'].values[0] ** .2)

19958400

In [25]:
# Test the answer
19958400 ** 5 == ans['product'].values[0]

True

# Part 3: Find the Number of Ways [Extra Credit]

Below is the 25 numbers would make the story happen.

In [26]:
right_25 = list(ans['comb'].values[0])
str(right_25)

'[6, 10, 12, 14, 15, 18, 20, 21, 22, 24, 28, 30, 33, 36, 40, 42, 44, 45, 48, 50, 54, 55, 56, 60, 66]'

Below are prime factors of one ticket. 

In [27]:
list(primefac.primefac(19958400))

[2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 5, 5, 7, 11]

## Step 1: Get all the combinations 

Get all combinations of the 5-number set out of these 25 numbers.  
There are 53130 of them.

In [28]:
len(list(combinations(right_25, 5)))

53130

The product of each combination should be 19958400.  
There are 275 combinations that are qualified.

In [29]:
combs_list = []
for comb in list(combinations(right_25, 5)):
    if get_product(comb) == 19958400:
        combs_list.append(comb)
combs_list = [list(x) for x in combs_list]
len(combs_list)

275

## Step 2: Use an inefficient algorithm to find all the possible ways

There are 274 ways.

In [30]:
successes = []
for comb in combs_list:
    result = comb
    #print(result)
    temp_11 = [x for x in combs_list if x != comb]
    temp_12 = [x for x in temp_11 if len(set(x).intersection(set(result))) == 0]
    if len(temp_12) != 0:
        
        for second_set in temp_12:
            result = result + second_set
            #print(result)
            temp_21 = [x for x in temp_12 if x != second_set]
            temp_22 = [x for x in temp_21 if len(set(x).intersection(set(result))) == 0]
            if len(temp_22) != 0:
                
                for third_set in temp_22:
                    result = result + third_set
                    #print(result)
                    temp_31 = [x for x in temp_22 if x != third_set]
                    temp_32 = [x for x in temp_31 if len(set(x).intersection(set(result))) == 0]
                    if len(temp_32) != 0:
                        
                        for fourth_set in temp_32:
                            result = result + fourth_set
                            #print(result)
                            temp_41 = [x for x in temp_32 if x != fourth_set]
                            temp_42 = [x for x in temp_41 if len(set(x).intersection(set(result))) == 0]
                            if len(temp_42) != 0:
                                result = result + temp_42[0]
                                successes.append(result)
                            else:
                                break
                                
len(successes)

274

Check if all the ways are correct.

In [31]:
# check
for success in successes:
    temp = [success[0+i*5:5+i*5] for i in range(5)]
    for bin_ in temp:
        if get_product(bin_) != 19958400:
            print('uh oh')

Below is one example

In [32]:
[[success[0+i*5:5+i*5] for i in range(5)] for success in successes][0]

[[6, 15, 56, 60, 66],
 [10, 14, 48, 54, 55],
 [12, 18, 42, 44, 50],
 [20, 21, 33, 36, 40],
 [22, 24, 28, 30, 45]]

## Step 3: Remove the duplicates

There is a need to remove duplicates since there could be identical 5 tickets in different order.

In [33]:
successes_adj = [sorted([success[0+i*5:5+i*5] for i in range(5)]) for success in successes]
successes_adj = [success[0] + success[1] + success[2] + success[3] + success[4] for success in successes_adj]
successes_adj = [', '.join([str(x) for x in success]) for success in successes_adj]

In [34]:
success_df = pd.DataFrame()
success_df['solutions'] = successes_adj
success_df.shape

(274, 1)

In [35]:
len(success_df['solutions'].unique())

244

So, there are 244 ways that the friends could have selected five numbers each so that all their statements are true.

Below are all the 244 possible ways.

In [36]:
way_num = 0
for way in success_df['solutions'].unique():
    way_num += 1
    way_adj = way.split(", ")
    buckets = [way_adj[0+5*i:5+5*i] for i in range(5)]
    print(f'Way {way_num}: ')
    for i in range(5):
        print(f'Ticket {i+1}: {[int(x) for x in buckets[i]]}')
    print()

Way 1: 
Ticket 1: [6, 15, 56, 60, 66]
Ticket 2: [10, 14, 48, 54, 55]
Ticket 3: [12, 18, 42, 44, 50]
Ticket 4: [20, 21, 33, 36, 40]
Ticket 5: [22, 24, 28, 30, 45]

Way 2: 
Ticket 1: [6, 18, 50, 56, 66]
Ticket 2: [10, 14, 44, 54, 60]
Ticket 3: [12, 15, 42, 48, 55]
Ticket 4: [20, 21, 33, 36, 40]
Ticket 5: [22, 24, 28, 30, 45]

Way 3: 
Ticket 1: [6, 18, 55, 56, 60]
Ticket 2: [10, 14, 40, 54, 66]
Ticket 3: [12, 20, 42, 44, 45]
Ticket 4: [15, 28, 30, 33, 48]
Ticket 5: [21, 22, 24, 36, 50]

Way 4: 
Ticket 1: [6, 20, 42, 60, 66]
Ticket 2: [10, 12, 54, 55, 56]
Ticket 3: [14, 15, 44, 45, 48]
Ticket 4: [18, 22, 28, 36, 50]
Ticket 5: [21, 24, 30, 33, 40]

Way 5: 
Ticket 1: [6, 20, 45, 56, 66]
Ticket 2: [10, 14, 44, 54, 60]
Ticket 3: [12, 15, 42, 48, 55]
Ticket 4: [18, 22, 28, 36, 50]
Ticket 5: [21, 24, 30, 33, 40]

Way 6: 
Ticket 1: [6, 20, 54, 55, 56]
Ticket 2: [10, 12, 42, 60, 66]
Ticket 3: [14, 15, 44, 45, 48]
Ticket 4: [18, 22, 28, 36, 50]
Ticket 5: [21, 24, 30, 33, 40]

Way 7: 
Ticket 1: [6, 